## **Fase 2 - Limpieza de Datos**
*En esta fase crucial del proceso ETL, la limpieza de datos se convierte en la base para un análisis confiable y preciso. La limpieza implica la eliminación de valores nulos, la corrección de incoherencias y el tratamiento de datos que podrían distorsionar los resultados. Este proceso garantiza que el conjunto de datos esté libre de ruido y preparado para extraer información valiosa. Al limpiar los datos, aseguramos que las conclusiones posteriores reflejen fielmente la realidad representada en los datos, estableciendo una base sólida sobre la cual se construirá nuestro análisis*.

In [1]:
# General
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_csv('df_completo.csv')

### **Análisis inicial del dataset para entender la estructura, los tipos de datos, los valores ausentes y las estadísticas generales**

In [3]:
df.head()

,urls,timestamp,record_id,Titles,Host_name,Property_types,Prices_per_night,Check_ins,Check_outs,Location,Ratings,Num_reviews,Cleaning_fee,Com_fee,Otros,Baños,Maximum_guests,Viajeros,Dormitorios,Camas
0,https://www.airbnb.es/rooms/125284963667401123...,2024-11-07 10:12:02,1252849636674011237,Reserva una (o más) camas en este dormitorio c...,Anfitrión: Generator,"Habitación en Barcelona, España",19 €,Horario de llegada: de 14:00 a 0:00,Salida antes de las 10:00,NaN,"4,75",4.0,0,0,1 litera,Baño compartido,NaN,NaN,NaN,NaN
1,https://www.airbnb.es/rooms/128244586043482024...,2024-11-07 10:12:02,1282445860434820248,Te damos la bienvenida a este elegante apartam...,Anfitrión: HostPro,"Alojamiento entero: apartamento en Barcelona, ...",110 €,Llegada a partir de las 15:00,Salida antes de las 11:00,"Barcelona, Catalunya, España",Nuevo,NaN,60 €,0,NaN,1 baño,NaN,4 viajeros,2 dormitorios,3 camas
2,https://www.airbnb.es/rooms/128028819924646861...,2024-11-07 10:12:02,1280288199246468615,Excelente y acogedor apartamento a solo '2 cam...,Quédate con Lautaro Gaspar,"Habitación en Barcelona, España",32 €,Llegada a partir de las 16:00,Salida antes de las 11:00,"Barcelona, Catalunya, España",Nuevo,NaN,0,0,NaN,Baño compartido,NaN,NaN,NaN,1 cama individual
3,https://www.airbnb.es/rooms/850684095339937556...,2024-11-07 10:12:02,850684095339937556,Desconecta de la rutina en este alojamiento a...,Anfitrión: Marta+Paul,"Alojamiento entero: apartamento en Barcelona, ...",80 €,Llegada a partir de las 15:00,Salida antes de las 11:00,NaN,"5,0",5.0,90 €,0,NaN,1 baño,NaN,2 viajeros,1 dormitorio,1 cama
4,https://www.airbnb.es/rooms/127904085557632410...,2024-11-07 10:12:02,1279040855576324107,Este anfitrión tiene 979 evaluaciones de otros...,Anfitrión: David,"Habitación en Barcelona, España",46 €,Horario de llegada: de 17:00 a 2:00,Salida antes de las 11:00,"Barcelona, Catalunya, España",Nuevo,NaN,15 €,42 €,NaN,Baño compartido,1.0,NaN,NaN,1 cama


In [4]:
df.shape

(2700, 20)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700 entries, 0 to 2699
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   urls              2700 non-null   object 
 1   timestamp         2700 non-null   object 
 2   record_id         2700 non-null   int64  
 3   Titles            2685 non-null   object 
 4   Host_name         2685 non-null   object 
 5   Property_types    2685 non-null   object 
 6   Prices_per_night  2573 non-null   object 
 7   Check_ins         2685 non-null   object 
 8   Check_outs        2685 non-null   object 
 9   Location          1626 non-null   object 
 10  Ratings           2643 non-null   object 
 11  Num_reviews       1500 non-null   float64
 12  Cleaning_fee      2700 non-null   object 
 13  Com_fee           2700 non-null   object 
 14  Otros             146 non-null    object 
 15  Baños             2618 non-null   object 
 16  Maximum_guests    2667 non-null   float64


In [6]:
df.describe()

,record_id,Num_reviews,Maximum_guests
count,2.700000e+03,1500.000000,2667.000000
mean,8.081992e+17,119.248000,2.693663
std,5.151280e+17,230.681026,1.615371
min,8.000400e+04,2.000000,1.000000
25%,5.138602e+07,15.000000,2.000000
50%,1.064768e+18,52.000000,2.000000
75%,1.238310e+18,136.000000,4.000000
max,1.283387e+18,2781.000000,13.000000


In [7]:
df.isna().sum()

urls                   0
timestamp              0
record_id              0
Titles                15
Host_name             15
Property_types        15
Prices_per_night     127
Check_ins             15
Check_outs            15
Location            1074
Ratings               57
Num_reviews         1200
Cleaning_fee           0
Com_fee                0
Otros               2554
Baños                 82
Maximum_guests        33
Viajeros            1280
Dormitorios         1247
Camas                116
dtype: int64

#### **Eliminación de columnas redundantes o duplicadas que no aportan valor al proceso de análisis**

In [8]:
df = df.drop(['Viajeros', 'Otros', 'Com_fee'], axis=1)

#### **Limpieza y Formateo de Columnas del DataFrame**

#### Titles

In [9]:
df['Titles'] = df['Titles'].str.capitalize()
# 2. Eliminar puntuación innecesaria
df['Titles'] = df['Titles'].str.replace('¡', '', regex=False)
# 3. Corregir espacios y saltos de línea
df['Titles'] = df['Titles'].str.replace(r'\s+', ' ', regex=True)
df['Titles'] = df['Titles'].str.strip()


#### Host

In [10]:
# Extraer solo el nombre del anfitrión, removendo "Anfitrión: " y cualquier outro prefixo
df['Host_name'] = df['Host_name'].str.replace('Anfitrión: ', '', regex=False).str.replace('Quédate con ', '', regex=False)

#### Property types

In [11]:
# Clasificamos los tipos de propiedad en tres categorías para extraer solo la información que queremos con una función
def category_types(value):
    if isinstance(value, float) and pd.isna(value):
        return "otro"
    elif "Alojamiento entero" in value:
        return "Alojamiento entero"
    elif "Habitación" in value:
        return "Habitación"
    else:
        return "otro"
df["Property_types"] = df["Property_types"].apply(category_types)    

#### Prices

In [12]:
# Remover el símbolo € y convertir a valores numéricos
df['Prices_per_night'] = pd.to_numeric(df['Prices_per_night'].str.replace('€', '').str.strip(), errors='coerce')
df = df.dropna(subset=['Prices_per_night']) # Mantener solo registros válidos para análisis

In [13]:
# Verificación después del dropna
df.shape

(2573, 17)

#### Check in/Check out

In [14]:
def process_check_times(df):

    def extract_check_in_hour(check_in):

        if isinstance(check_in, str):
            check_in = check_in.strip().lower()

            if 'llegada' in check_in:
                # Elimina la parte "llegada" y obtiene la primera hora válida
                check_in = check_in.replace('llegada:', '')
                check_in = check_in.replace('de', '')
                parts = check_in.split()
                for part in parts:
                    if ":" in part:  # Verifica si el elemento tiene formato de hora "HH:MM"
                        return part.strip()

        return np.nan

    def extract_check_out_hour(check_out):

        if isinstance(check_out, str) and "antes de las" in check_out:
            try:  # Extrae solo la hora
                hour_str = check_out.split("antes de las")[1].split(":")[0].strip() + ":00"  # Devuelve la hora en formato "HH:00"
                return hour_str
            except:
                return np.nan
        return np.nan

    # Función para calcular las horas restantes hasta el check-in
    def calculate_hours_to_check_in(check_in_hour):
        try:
            # Extraímos la hora directamente de la cadena
            hour = int(check_in_hour.split(":")[0])  # Obtiene la hora (antes de los minutos)

            # Verifica si la hora es válida
            if 0 <= hour < 24:
                return 24 - hour  # Calcula las horas restantes hasta el check-in
        except Exception:
            return np.nan

        return np.nan

    # Aplica las funciones a las columnas correspondientes
    df['check_in_hour'] = df['Check_ins'].apply(extract_check_in_hour)
    df['check_out_hour'] = df['Check_outs'].apply(extract_check_out_hour)
    df['total_hours_checkin'] = df['check_in_hour'].apply(calculate_hours_to_check_in)

    df = df.drop(columns=['Check_ins', 'Check_outs'])

    return df


In [15]:
df = process_check_times(df)

#### Cleaning Fees

In [16]:
df['Cleaning_fee'] = pd.to_numeric(df['Cleaning_fee'].str.replace('€', '').str.strip(), errors='coerce')

#### Ratings

In [17]:
# Sustituir valores "Nuevo" y "Sin evaluaciones" por 0
df['Ratings'] = df['Ratings'].replace({'Nuevo': '0', 'Sin evaluaciones': '0'})

# Eliminar comas y convertir a numérico
df['Ratings'] = pd.to_numeric(df['Ratings'].str.replace(',', '.'), errors='coerce').fillna(0)

#### Number of reviews

In [18]:
# Convertir a numérico y reemplazar valores nulos con 0
df['Num_reviews'] = pd.to_numeric(df['Num_reviews'], errors='coerce').fillna(0)

#### Dormitorios

In [19]:
# Extraer solo el número de dormitorios eliminando letras
df['Dormitorios'] = df['Dormitorios'].str.extract('(\d+)')[0]

df['Dormitorios'] = pd.to_numeric(df['Dormitorios'], errors='coerce')

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\diogo\AppData\Local\Temp\ipykernel_26496\3909473064.py:2: SyntaxWarning: invalid escape sequence '\d'
  df['Dormitorios'] = df['Dormitorios'].str.extract('(\d+)')[0]


#### Camas

In [20]:
# Extraer solo el número de camas eliminando letras
df['Camas'] = df['Camas'].str.extract('(\d+)')[0]

df['Camas'] = pd.to_numeric(df['Camas'], errors='coerce')

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\diogo\AppData\Local\Temp\ipykernel_26496\1413491753.py:2: SyntaxWarning: invalid escape sequence '\d'
  df['Camas'] = df['Camas'].str.extract('(\d+)')[0]


#### Baños

In [21]:
# Reemplazar "Baño compartido" por 0.5 y "Baño privado" por 1
df['Baños'] = df['Baños'].replace({'Baño compartido': 0.5, 'Baño privado': 1})

# Extraer números eliminando letras
df['Baños'] = df['Baños'].str.extract('(\d+[.,]?\d*)')[0]# El resultado es una Serie con el número y usamos '[0]' para seleccionar esa columna

df['Baños'] = pd.to_numeric(df['Baños'].str.replace(',', '.'), errors='coerce')

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\diogo\AppData\Local\Temp\ipykernel_26496\280671281.py:5: SyntaxWarning: invalid escape sequence '\d'
  df['Baños'] = df['Baños'].str.extract('(\d+[.,]?\d*)')[0]# El resultado es una Serie con el número y usamos '[0]' para seleccionar esa columna


In [ ]:
df.to_csv('df_final_cleaned.csv', index=False)